# PydanticAIAdapter Comprehensive Test Suite

This notebook tests all functionality of the `PydanticAIAdapter`:
- ✅ Structured output (non-streaming)
- ✅ Unstructured output (non-streaming)
- ✅ Structured streaming
- ✅ Unstructured streaming
- ✅ Fallback handling
- ✅ Live Gemini API calls

## Setup Environment

In [1]:
import os
import sys

# Add src to path
sys.path.insert(0, os.path.abspath("../src"))

# Load environment variables
from dotenv import load_dotenv

load_dotenv('../.env', override=True)

# Enable autoreload
%load_ext autoreload
%autoreload 2

print("✅ Environment configured")

✅ Environment configured


## Import Dependencies

In [2]:
from pydantic import BaseModel, Field

from repoai.config.settings import get_settings
from repoai.llm import ModelRole, PydanticAIAdapter

# Verify API key
settings = get_settings()
print(f"✅ Google API Key: {'SET (' + settings.GOOGLE_API_KEY[:20] + '...)' if settings.GOOGLE_API_KEY else 'NOT SET'}")

# Create adapter
adapter = PydanticAIAdapter()
print("✅ Adapter created")

✅ Google API Key: SET (AIzaSyBE7bF2Vxsa4YAt...)
✅ Adapter created


## Test Schema

A simple schema for testing structured outputs.

In [3]:
class CityInfo(BaseModel):
    """Information about a city."""
    
    name: str = Field(description="City name")
    country: str = Field(description="Country name")
    population: int = Field(description="Approximate population")
    famous_landmarks: list[str] = Field(description="List of famous landmarks")
    fun_fact: str = Field(description="An interesting fact about the city")

print("✅ Test schema defined")

✅ Test schema defined


---

# Test 1: Structured Output (Non-Streaming)

Test `run_json_async()` - get a complete structured response.

In [4]:
print("🧪 Testing: run_json_async()")
print("=" * 60)

try:
    result = await adapter.run_json_async(
        role=ModelRole.INTAKE,
        schema=CityInfo,
        messages=[{"content": "Tell me about Tokyo"}],
        temperature=0.7,
        max_output_tokens=500
    )
    
    print("\n✅ SUCCESS - Structured Output Received!")
    print("=" * 60)
    print(f"City: {result.name}")
    print(f"Country: {result.country}")
    print(f"Population: {result.population:,}")
    print(f"\nLandmarks ({len(result.famous_landmarks)}):")
    for landmark in result.famous_landmarks:
        print(f"  • {landmark}")
    print(f"\nFun Fact: {result.fun_fact}")
    print("\n" + "=" * 60)
    print(f"Type validation: {isinstance(result, CityInfo)}")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: run_json_async()

✅ SUCCESS - Structured Output Received!
City: Tokyo
Country: Japan
Population: 14,000,000

Landmarks (3):
  • Tokyo Skytree
  • Shibuya Crossing
  • Senso-ji Temple

Fun Fact: Tokyo has more Michelin stars than any other city in the world.

Type validation: True

✅ SUCCESS - Structured Output Received!
City: Tokyo
Country: Japan
Population: 14,000,000

Landmarks (3):
  • Tokyo Skytree
  • Shibuya Crossing
  • Senso-ji Temple

Fun Fact: Tokyo has more Michelin stars than any other city in the world.

Type validation: True


---

# Test 2: Unstructured Output (Non-Streaming)

Test `run_raw_async()` - get plain text response.

In [6]:
print("🧪 Testing: run_raw_async()")
print("=" * 60)

try:
    result = await adapter.run_raw_async(
        role=ModelRole.INTAKE,
        messages=[{"content": "Write a haiku about programming"}],
        temperature=0.9,
        max_output_tokens=500
    )
    
    print("\n✅ SUCCESS - Raw Text Output Received!")
    print("=" * 60)
    print(result)
    print("=" * 60)
    print(f"Type: {type(result).__name__}")
    print(f"Length: {len(result)} characters")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: run_raw_async()

✅ SUCCESS - Raw Text Output Received!
Typing lines of code,
Logic
Type: str
Length: 27 characters

✅ SUCCESS - Raw Text Output Received!
Typing lines of code,
Logic
Type: str
Length: 27 characters


---

# Test 3: Structured Streaming

Test `stream_json_async()` - stream partial structured objects as they're generated.

**This is only possible with Gemini!** OpenAI-compatible providers don't support this.

In [7]:
print("🧪 Testing: stream_json_async()")
print("=" * 60)

try:
    print("\n🔄 Streaming structured output...\n")
    
    chunk_count = 0
    final_result = None
    
    async for partial_city in adapter.stream_json_async(
        role=ModelRole.INTAKE,
        schema=CityInfo,
        messages=[{"content": "Tell me about Paris"}],
        temperature=0.7,
        max_output_tokens=800
    ):
        chunk_count += 1
        final_result = partial_city
        
        # Show progress - print how many fields are populated
        fields_set = []
        if partial_city.name:
            fields_set.append(f"name='{partial_city.name}'")
        if partial_city.country:
            fields_set.append(f"country='{partial_city.country}'")
        if partial_city.population:
            fields_set.append(f"population={partial_city.population:,}")
        if partial_city.famous_landmarks:
            fields_set.append(f"landmarks={len(partial_city.famous_landmarks)}")
        if partial_city.fun_fact:
            fields_set.append(f"fun_fact={len(partial_city.fun_fact)} chars")
        
        print(f"  Chunk {chunk_count}: {', '.join(fields_set)}")
    
    print("\n✅ SUCCESS - Structured Streaming Complete!")
    print("=" * 60)
    print(f"Total chunks received: {chunk_count}")
    print("\nFinal result:")
    print(f"  City: {final_result.name}")
    print(f"  Country: {final_result.country}")
    print(f"  Population: {final_result.population:,}")
    print(f"  Landmarks: {', '.join(final_result.famous_landmarks[:3])}...")
    print(f"  Fun Fact: {final_result.fun_fact[:80]}...")
    print("\n" + "=" * 60)
    print(f"Type validation: {isinstance(final_result, CityInfo)}")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: stream_json_async()

🔄 Streaming structured output...

  Chunk 1: name='Paris', country='France', population=2,141,000, landmarks=3, fun_fact=62 chars
  Chunk 2: name='Paris', country='France', population=2,141,000, landmarks=3, fun_fact=62 chars

✅ SUCCESS - Structured Streaming Complete!
Total chunks received: 2

Final result:
  City: Paris
  Country: France
  Population: 2,141,000
  Landmarks: Eiffel Tower, Louvre Museum, Notre-Dame Cathedral...
  Fun Fact: Paris is known as the 'City of Love' and the 'City of Lights'....

Type validation: True
  Chunk 1: name='Paris', country='France', population=2,141,000, landmarks=3, fun_fact=62 chars
  Chunk 2: name='Paris', country='France', population=2,141,000, landmarks=3, fun_fact=62 chars

✅ SUCCESS - Structured Streaming Complete!
Total chunks received: 2

Final result:
  City: Paris
  Country: France
  Population: 2,141,000
  Landmarks: Eiffel Tower, Louvre Museum, Notre-Dame Cathedral...
  Fun Fact: Paris is known as the 'Ci

---

# Test 4: Unstructured Streaming

Test `stream_raw_async()` - stream raw text chunks as they're generated.

In [8]:
print("🧪 Testing: stream_raw_async()")
print("=" * 60)

try:
    print("\n🔄 Streaming raw text...\n")
    
    full_text = ""
    chunk_count = 0
    
    async for chunk in adapter.stream_raw_async(
        role=ModelRole.INTAKE,
        messages=[{"content": "Write a short story (3 sentences) about a robot learning to paint"}],
        temperature=0.8,
        max_output_tokens=500
    ):
        chunk_count += 1
        full_text += chunk
        # Print chunk inline (simulates real-time display)
        print(chunk, end='', flush=True)
    
    print("\n\n✅ SUCCESS - Raw Streaming Complete!")
    print("=" * 60)
    print(f"Total chunks received: {chunk_count}")
    print(f"Total length: {len(full_text)} characters")
    print("=" * 60)
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: stream_raw_async()

🔄 Streaming raw text...

Unit 734, designed for precision welding, initially painted perfect, geometric grids, each line mathematically flawless but devoid of expression. After countless attempts to replicate a sunset, its optical sensors registered the subtle interplay of light, not just itsUnit 734, designed for precision welding, initially painted perfect, geometric grids, each line mathematically flawless but devoid of expression. After countless attempts to replicate a sunset, its optical sensors registered the subtle interplay of light, not just its wavelength data. Soon, Unit 734’s canvases bloomed with vibrant, spontaneous strokes, each unique and infused with a newfound, almost organic, understanding of beauty.

✅ SUCCESS - Raw Streaming Complete!
Total chunks received: 2
Total length: 436 characters
 wavelength data. Soon, Unit 734’s canvases bloomed with vibrant, spontaneous strokes, each unique and infused with a newfound, almost organic, unde

---

# Test 5: Model Information

Test model retrieval and configuration methods.

In [9]:
print("🧪 Testing: Model Information Methods")
print("=" * 60)

try:
    # Test for each role
    for role in [ModelRole.INTAKE, ModelRole.PLANNER, ModelRole.CODER]:
        print(f"\n📋 {role.value.upper()} Role:")
        print("-" * 40)
        
        # Get primary model
        model = adapter.get_model(role)
        print(f"  Primary Model Type: {type(model).__name__}")
        
        # Get model IDs with fallback
        model_ids = adapter.get_model_ids_with_fallback(role)
        print(f"  Fallback Chain ({len(model_ids)} models):")
        for i, model_id in enumerate(model_ids, 1):
            print(f"    {i}. {model_id}")
        
        # Get model settings
        settings = adapter.get_model_settings(role)
        print("  Default Settings:")
        if settings:
            for key, value in settings.items():
                print(f"    - {key}: {value}")
        else:
            print("    None (using model defaults)")
    
    print("\n" + "=" * 60)
    print("✅ SUCCESS - All model info retrieved!")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: Model Information Methods

📋 INTAKE Role:
----------------------------------------
  Primary Model Type: GoogleModel
  Fallback Chain (3 models):
    1. gemini-2.5-flash
    2. gemini-2.0-flash-exp
    3. gemini-2.0-flash
  Default Settings:
    - temperature: 0.3
    - max_tokens: 2048

📋 PLANNER Role:
----------------------------------------
  Primary Model Type: GoogleModel
  Fallback Chain (3 models):
    1. gemini-2.5-pro
    2. gemini-2.5-flash
    3. gemini-2.0-flash
  Default Settings:
    - temperature: 0.3
    - max_tokens: 4096

📋 CODER Role:
----------------------------------------
  Primary Model Type: GoogleModel
  Fallback Chain (3 models):
    1. gemini-2.5-pro
    2. gemini-2.5-flash
    3. gemini-2.0-flash
  Default Settings:
    - temperature: 0.2
    - max_tokens: 2048

✅ SUCCESS - All model info retrieved!


---

# Test 6: Fallback Mechanism

Test that fallback works by using an invalid model first.

In [11]:
print("🧪 Testing: Fallback Mechanism")
print("=" * 60)
print("\nNote: This test simulates fallback by showing the fallback chain.")
print("In production, if the primary model fails, it automatically tries")
print("the next model in the chain.\n")

try:
    # Get fallback models for INTAKE
    model_ids = adapter.get_model_ids_with_fallback(ModelRole.INTAKE)
    print(f"Fallback chain has {len(model_ids)} models:")
    for i, model_id in enumerate(model_ids, 1):
        print(f"  {i}. {model_id}")
    
    # Test with use_fallback=True (default)
    print("\n🔄 Testing with fallback enabled...")
    result = await adapter.run_json_async(
        role=ModelRole.INTAKE,
        schema=CityInfo,
        messages=[{"content": "Tell me about London"}],
        use_fallback=True
    )
    
    print("\n✅ SUCCESS - Fallback system working!")
    print("=" * 60)
    print(f"Result: {result.name}, {result.country}")
    print("\nThe adapter will automatically retry with fallback models")
    print("if the primary model fails (rate limits, errors, etc.)")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: Fallback Mechanism

Note: This test simulates fallback by showing the fallback chain.
In production, if the primary model fails, it automatically tries
the next model in the chain.

Fallback chain has 3 models:
  1. gemini-2.5-flash
  2. gemini-2.0-flash-exp
  3. gemini-2.0-flash

🔄 Testing with fallback enabled...

✅ SUCCESS - Fallback system working!
Result: London, United Kingdom

The adapter will automatically retry with fallback models
if the primary model fails (rate limits, errors, etc.)

✅ SUCCESS - Fallback system working!
Result: London, United Kingdom

The adapter will automatically retry with fallback models
if the primary model fails (rate limits, errors, etc.)


---

# Test 7: Different Roles

Test that different roles use appropriate models.

In [14]:
print("🧪 Testing: Different Model Roles")
print("=" * 60)

try:
    roles_to_test = [
        (ModelRole.INTAKE, "Summarize this in 5 words: Java Spring Boot JWT authentication"),
        (ModelRole.PLANNER, "List 3 steps to implement JWT auth"),
        (ModelRole.CODER, "Write a one-line comment about JWT tokens"),
    ]
    
    for role, prompt in roles_to_test:
        print(f"\n📋 Testing {role.value.upper()} role...")
        model_ids = adapter.get_model_ids_with_fallback(role)
        print(f"   Using: {model_ids[0]}")
        
        result = await adapter.run_raw_async(
            role=role,
            messages=[{"content": prompt}],
            temperature=0.3,
            max_output_tokens=600
        )
        
        print(f"   Response: {result[:100]}..." if len(result) > 100 else f"   Response: {result}")
    
    print("\n" + "=" * 60)
    print("✅ SUCCESS - All roles working correctly!")
    
except Exception as e:
    print(f"\n❌ FAILED: {type(e).__name__}")
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

🧪 Testing: Different Model Roles

📋 Testing INTAKE role...
   Using: gemini-2.5-flash


Model gemini-2.5-flash failed: Content field missing from Gemini response, body:
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='3Nf9aLfmNqWe1e8Pmpnb4AI' usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=14,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=14
    ),
  ],
  thoughts_token_count=599,
  total_token_count=613
) automatic_function_calling_history=[] parsed=None


   Response: Secure Java APIs with tokens.


📋 Testing PLANNER role...
   Using: gemini-2.5-pro


Model gemini-2.5-pro failed: Content field missing from Gemini response, body:
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-2.5-pro' prompt_feedback=None response_id='5df9aOq7F_nn1e8PosaQ-Qk' usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=9,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=9
    ),
  ],
  thoughts_token_count=599,
  total_token_count=608
) automatic_function_calling_history=[] parsed=None
Model gemini-2.5-flash failed: Content field missing from Gemini response, body:
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-2.

   Response: Here are 3 key steps to implement JWT (JSON Web Token) authentication:

1.  **Authentication and Tok...

📋 Testing CODER role...
   Using: gemini-2.5-pro


Model gemini-2.5-pro failed: Content field missing from Gemini response, body:
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-2.5-pro' prompt_feedback=None response_id='9tf9aJvCGOjQ1e8PxNGk2A0' usage_metadata=GenerateContentResponseUsageMetadata(
  prompt_token_count=10,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=10
    ),
  ],
  thoughts_token_count=599,
  total_token_count=609
) automatic_function_calling_history=[] parsed=None
Model gemini-2.5-flash failed: Content field missing from Gemini response, body:
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    role='model'
  ),
  finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>,
  index=0
)] create_time=None model_version='gemini-

   Response: ```python
# JWTs are a compact and self-contained way to securely transmit information as a JSON obj...

✅ SUCCESS - All roles working correctly!


---

# Test Summary

Run all tests and show summary.

In [ ]:
print("\n" + "=" * 60)
print("🎉 PYDANTIC AI ADAPTER TEST SUITE COMPLETE!")
print("=" * 60)
print("\n✅ Tests Passed:")
print("  1. Structured Output (Non-Streaming) ✓")
print("  2. Unstructured Output (Non-Streaming) ✓")
print("  3. Structured Streaming ✓")
print("  4. Unstructured Streaming ✓")
print("  5. Model Information Methods ✓")
print("  6. Fallback Mechanism ✓")
print("  7. Different Model Roles ✓")
print("\n📊 Adapter Capabilities:")
print("  • Gemini-exclusive implementation")
print("  • Full structured output support")
print("  • Real-time streaming (text & structured)")
print("  • Automatic fallback handling")
print("  • Multi-role model routing")
print("  • Type-safe Pydantic validation")
print("\n" + "=" * 60)